<a href="https://colab.research.google.com/github/good-thinking/easy-coding2/blob/main/%EA%B1%B4%EC%84%A4%EC%9E%A5%EB%B9%84%EC%98%A4%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install catboost
!pip install optuna

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

from scipy.stats import ranksums

from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_absolute_error, roc_auc_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostClassifier, CatBoostRegressor

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

import random
from collections import Counter

import pickle

import warnings
warnings.filterwarnings(action = 'ignore')

!pip list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
alembic                       1.9.2
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd               

In [ ]:
!pip list

dftrain=pd.read_csv("/content/drive/MyDrive/안녕/open/train.csv")
dftest=pd.read_csv("/content/drive/MyDrive/안녕/open/test.csv")
submission=pd.read_csv("/content/drive/MyDrive/안녕/open/sample_submission.csv")


Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
alembic                       1.9.2
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
autopage                      0.5.1
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis  

In [ ]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  object 
 2   ANONYMOUS_1          14095 non-null  int64  
 3   YEAR                 14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY  14095 non-null  int64  
 5   ANONYMOUS_2          14095 non-null  int64  
 6   AG                   14095 non-null  int64  
 7   AL                   14095 non-null  int64  
 8   B                    14095 non-null  int64  
 9   BA                   14095 non-null  int64  
 10  BE                   14095 non-null  int64  
 11  CA                   14095 non-null  int64  
 12  CD                   12701 non-null  float64
 13  CO                   14095 non-null  int64  
 14  CR                   14095 non-null  int64  
 15  CU                   14095 non-null 

In [ ]:
dftrain.isnull().sum()

ID                         0
COMPONENT_ARBITRARY        0
ANONYMOUS_1                0
YEAR                       0
SAMPLE_TRANSFER_DAY        0
ANONYMOUS_2                0
AG                         0
AL                         0
B                          0
BA                         0
BE                         0
CA                         0
CD                      1394
CO                         0
CR                         0
CU                         0
FH2O                   10205
FNOX                   10205
FOPTIMETHGLY           10205
FOXID                  10205
FSO4                   10205
FTBN                   10205
FE                         0
FUEL                   10205
H2O                        0
K                       2299
LI                         0
MG                         0
MN                         0
MO                         0
NA                         0
NI                         0
P                          0
PB                         0
PQINDEX       

In [ ]:
dftrain.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')

In [ ]:
dftrain.iloc[:,1:7].describe()

,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG
count,14095.000000,14095.000000,14095.000000,14095.000000,14095.000000
mean,3146.082937,2013.652501,7.600568,387.416885,0.025825
std,4216.089809,3.964758,11.681628,550.016073,0.171926
min,1000.000000,2007.000000,0.000000,200.000000,0.000000
25%,1655.000000,2010.000000,3.000000,200.000000,0.000000
50%,2227.000000,2014.000000,5.000000,200.000000,0.000000
75%,3797.000000,2017.000000,8.000000,410.000000,0.000000
max,294451.000000,2022.000000,368.000000,9650.000000,3.000000


In [ ]:
dftest.iloc[:,1:53].describe()

,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
count,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000,6041.000000
mean,3199.140871,2013.656514,392.669922,0.026817,0.030790,3.155438,37.397285,198.615295,0.024599,3.127462,25.042212,0.883463,475.091210,0.797219,0.044529,110.495135,581.186393
std,4834.849930,3.976759,563.705290,0.179981,0.268141,21.120731,153.922102,690.985612,0.335461,11.315785,62.245538,5.708857,1812.652841,7.366433,0.359595,45.381679,530.799040
min,1000.000000,2007.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.900000,1.000000
25%,1667.000000,2010.000000,200.000000,0.000000,0.000000,0.000000,1.000000,15.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,73.000000,38.000000
50%,2271.000000,2014.000000,200.000000,0.000000,0.000000,1.000000,4.000000,41.000000,0.000000,1.000000,1.000000,0.000000,30.000000,0.000000,0.000000,112.100000,489.000000
75%,3842.000000,2017.000000,417.000000,0.000000,0.000000,2.000000,17.000000,143.000000,0.000000,2.000000,8.000000,0.000000,173.000000,0.000000,0.000000,138.200000,1115.000000
max,239570.000000,2022.000000,9650.000000,2.000000,13.000000,1415.000000,4113.000000,30407.000000,15.200000,472.000000,765.000000,155.000000,48976.000000,280.000000,13.000000,388.400000,1932.000000


In [ ]:
dftrain.iloc[:,:].isna().sum()

ID                         0
COMPONENT_ARBITRARY        0
ANONYMOUS_1                0
YEAR                       0
SAMPLE_TRANSFER_DAY        0
ANONYMOUS_2                0
AG                         0
AL                         0
B                          0
BA                         0
BE                         0
CA                         0
CD                      1394
CO                         0
CR                         0
CU                         0
FH2O                   10205
FNOX                   10205
FOPTIMETHGLY           10205
FOXID                  10205
FSO4                   10205
FTBN                   10205
FE                         0
FUEL                   10205
H2O                        0
K                       2299
LI                         0
MG                         0
MN                         0
MO                         0
NA                         0
NI                         0
P                          0
PB                         0
PQINDEX       

In [ ]:
mis_1 = dftrain.isnull().sum()
mis_1_1 = mis_1 >= 1
mis_1_df = pd.concat([mis_1, mis_1_1], axis = 1)
mis_1_df = mis_1_df.rename(columns = {0 : 'mis_1', 1 : 'mis_1_1'})

print("결측이 있는 변수")
print(mis_1_df['mis_1_1'].value_counts())

결측이 있는 변수
False    35
True     19
Name: mis_1_1, dtype: int64


In [ ]:
mis_1_data = mis_1_df.loc[mis_1_df['mis_1_1'] == True, :]
mis_1_data['ratio'] = mis_1_data['mis_1'] / 14095
mis_1_data

,mis_1,mis_1_1,ratio
CD,1394,True,0.098900
FH2O,10205,True,0.724016
FNOX,10205,True,0.724016
FOPTIMETHGLY,10205,True,0.724016
FOXID,10205,True,0.724016
FSO4,10205,True,0.724016
FTBN,10205,True,0.724016
FUEL,10205,True,0.724016
K,2299,True,0.163107
SOOTPERCENTAGE,10205,True,0.724016


In [ ]:
notnull_columns = dftrain.loc[:, dftrain.notnull().sum(axis = 0) == 14095].columns
notnull_columns = notnull_columns[1 :]
notnull_columns = list(notnull_columns)

select_columns = notnull_columns[: 32] + ['K', 'CD'] + notnull_columns[32 :]
dftrain2 = dftrain.loc[:, select_columns]
print(dftrain2.shape)
print(dftrain2.columns)

(14095, 36)
Index(['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'SAMPLE_TRANSFER_DAY',
       'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CO', 'CR', 'CU',
       'FE', 'H2O', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI', 'P', 'PB', 'PQINDEX',
       'S', 'SB', 'SI', 'SN', 'TI', 'V', 'V40', 'K', 'CD', 'ZN', 'Y_LABEL'],
      dtype='object')


In [ ]:
t = dftrain2.copy()
corr = t.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,CR,CU,FE,H2O,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,TI,V,V40,K,CD,ZN,Y_LABEL
ANONYMOUS_1,1.000000,0.106546,-0.041395,0.072154,-0.025674,0.017924,0.013467,-0.064204,-0.010805,-0.015321,-0.003783,-0.007035,-0.014086,0.000440,0.003868,0.000944,-0.001807,-0.004212,-0.006206,0.000183,-0.007862,-0.013647,-0.000499,0.002277,0.016374,-0.011797,0.005141,-0.009564,0.001832,0.001883,0.019542,-0.004384,0.010076,-0.019991,0.044197
YEAR,0.106546,1.000000,-0.094086,0.138199,-0.129124,-0.017525,0.011145,-0.098589,-0.063859,0.090465,-0.052337,-0.028871,-0.137593,-0.057709,0.009855,0.002697,0.025616,-0.039132,-0.091482,0.028953,-0.049206,-0.143675,-0.031021,-0.067160,-0.141084,-0.019612,0.011831,-0.065335,0.006399,-0.027624,-0.051584,-0.032801,-0.012668,0.048572,-0.087871
SAMPLE_TRANSFER_DAY,-0.041395,-0.094086,1.000000,-0.000060,0.021211,-0.017493,0.004305,0.016909,0.009600,0.008826,0.002195,-0.001495,0.006303,0.011651,-0.007926,-0.002275,0.008802,0.006816,0.001051,-0.006263,0.015599,-0.009730,0.000543,0.033622,0.014327,-0.015647,0.007713,0.006986,-0.002855,-0.002412,0.013896,0.012344,-0.006108,-0.019428,-0.004315
ANONYMOUS_2,0.072154,0.138199,-0.000060,1.000000,-0.006266,-0.022788,-0.100307,-0.011871,0.011162,-0.043046,-0.000070,-0.001826,-0.002244,-0.004608,-0.003760,-0.002483,-0.003306,0.003304,-0.008465,0.000647,-0.003914,0.000771,0.000519,-0.010409,-0.033153,0.006001,0.007633,0.004799,-0.000981,-0.006469,-0.025339,-0.003246,0.012702,0.032758,-0.033641
AG,-0.025674,-0.129124,0.021211,-0.006266,1.000000,0.019600,-0.028131,0.031649,0.020884,-0.004489,0.008861,0.005397,0.051046,0.025959,-0.003850,0.002394,-0.009523,0.019507,0.013124,-0.007197,0.054014,0.024517,0.020716,0.030929,0.017271,0.028327,0.020739,0.016582,0.003609,-0.003853,0.013592,0.013082,0.004325,0.002631,0.014671
AL,0.017924,-0.017525,-0.017493,-0.022788,0.019600,1.000000,0.000037,0.240863,-0.000858,-0.070322,0.009920,0.011430,0.037530,0.051138,-0.000093,0.000008,-0.001625,0.033695,-0.002615,-0.002290,0.099946,-0.022301,-0.004914,0.030492,0.011343,0.003998,0.087274,-0.006664,0.062613,0.026213,0.013823,0.037586,-0.004580,-0.024873,0.370512
B,0.013467,0.011145,0.004305,-0.100307,-0.028131,0.000037,1.000000,-0.045454,-0.013252,0.356121,0.005536,0.003759,0.000142,0.003075,-0.005442,0.015888,0.003456,0.000792,-0.001100,0.004937,0.010686,-0.004137,0.008009,-0.003177,0.005926,-0.019423,0.004165,-0.000990,0.008712,0.018197,-0.000882,-0.004724,-0.018408,-0.006463,-0.029787
BA,-0.064204,-0.098589,0.016909,-0.011871,0.031649,0.240863,-0.045454,1.000000,0.029900,-0.008642,0.004970,0.000443,0.023212,0.012627,-0.005055,-0.005337,-0.006470,0.007149,-0.004351,-0.004076,0.018465,0.003426,0.000497,0.003411,0.013244,0.006005,0.010588,0.004767,0.005074,0.000480,0.004150,0.007964,-0.001110,-0.009574,0.104840
BE,-0.010805,-0.063859,0.009600,0.011162,0.020884,-0.000858,-0.013252,0.029900,1.000000,0.006735,0.014084,0.002712,-0.001388,0.002707,-0.001610,-0.000977,0.045744,-0.000759,0.000208,-0.005445,0.001227,0.024291,0.006927,-0.000746,0.002929,0.007266,-0.002948,0.025744,-0.002501,0.000482,0.001359,0.003115,-0.002690,0.012394,0.010685
CA,-0.015321,0.090465,0.008826,-0.043046,-0.004489,-0.070322,0.356121,-0.008642,0.006735,1.000000,0.001185,-0.015970,0.008900,-0.012488,-0.013057,-0.003960,0.023333,0.011866,-0.001413,0.014019,-0.003472,0.032399,0.022724,-0.003129,-0.078747,0.022265,-0.006390,-0.006153,-0.013781,-0.014419,-0.059102,-0.007404,-0.005770,0.110625,-0.150379


In [ ]:
corr["AL"].sort_values()

In [ ]:
corr.loc[corr["Y_LABEL"] > 0.3, "Y_LABEL"]

AL         0.370512
Y_LABEL    1.000000
Name: Y_LABEL, dtype: float64

In [ ]:
corr.loc[corr["CD"] > 0.3, "CD"]

CD    1.0
Name: CD, dtype: float64

In [ ]:
corr.loc[corr["K"] > 0.3, "K"]

FE    0.310823
MN    0.510543
NA    0.316192
NI    0.341336
SI    0.605120
TI    0.524070
K     1.000000
Name: K, dtype: float64

In [ ]:
corr.loc[corr["SI"] > 0.3, "SI"]

CO         0.360847
FE         0.649188
MN         0.554424
NI         0.630178
PQINDEX    0.448144
SI         1.000000
TI         0.605636
K          0.605120
Name: SI, dtype: float64

In [ ]:
corr.loc[corr["FE"] > 0.3, "FE"]

CO         0.536765
CR         0.314226
FE         1.000000
MN         0.622415
NI         0.583322
PQINDEX    0.427582
SI         0.649188
TI         0.384018
V          0.341445
K          0.310823
Name: FE, dtype: float64

In [ ]:
corr.loc[corr["MN"] > 0.3, "MN"]

CO         0.425149
FE         0.622415
MN         1.000000
NI         0.527618
PQINDEX    0.376253
SI         0.554424
TI         0.585042
V          0.375177
K          0.510543
Name: MN, dtype: float64

In [ ]:
corr.loc[corr["TI"] > 0.3, "TI"]

FE    0.384018
MN    0.585042
NI    0.413859
SI    0.605636
TI    1.000000
V     0.677756
K     0.524070
Name: TI, dtype: float64

In [ ]:
corr.loc[corr["V"] > 0.3, "V"]

CR    0.391048
FE    0.341445
MN    0.375177
TI    0.677756
V     1.000000
Name: V, dtype: float64

In [ ]:
def IQR_outlier(data) :
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - (1.5 * IQR) 
    upper_bound = Q3 + (1.5 * IQR)

    return pd.concat([lower_bound, upper_bound], axis = 1).T


In [ ]:


ranksum_p = []
variable = ['AL','MN','FE',"K", 'SI', 'TI']

# alternative = 'greater'을 걸어줘서 한쪽 검정으로 실시
# scipy 1.8.1 버전 이상이어야 ranksums의 옵션인 alternative(한쪽 검정) 사용 가능
for v in variable :
  temp = ranksums(dftrain2.loc[(dftrain2['Y_LABEL'] == 1) & (dftrain2[v].notnull()), v], 
                  dftrain2.loc[(dftrain2['Y_LABEL'] == 0) & (dftrain2[v].notnull()), v],
                  alternative = 'greater').pvalue
  ranksum_p.append(temp)

Wilcoxon_var_table = pd.DataFrame({'variable' : variable,
                                   'p_value' : ranksum_p,
                                   'p_value_round' : np.round(ranksum_p, 4)})
Wilcoxon_var_table.sort_values('p_value')   

,variable,p_value,p_value_round
0,AL,2.219279e-275,0.0000
1,MN,8.715576e-08,0.0000
2,FE,5.957981e-07,0.0000
3,K,1.367787e-04,0.0001
4,SI,2.850652e-03,0.0029
5,TI,2.759862e-02,0.0276


In [ ]:
variable = ['AL','MN','FE',"K", 'SI', 'TI']

outlier_table = pd.DataFrame(np.zeros([8, len(variable)]), columns = variable,
                             index = ['정상 범위 데이터 개수', '이상 범위 데이터 개수', '전체 데이터 합계', 
                                      '정상 범위 데이터의 불량 개수', '이상 범위 데이터의 불량 개수', '불량 합계',
                                      '정상 범위 데이터의 불량률', '이상 범위 데이터의 불량률'])

for v in variable :
  temp1 = dftrain2.loc[(IQR_outlier(dftrain2).loc[0, v] <= dftrain2[v]) & 
                        (dftrain2[v] <= IQR_outlier(dftrain2).loc[1, v]), :]
  temp2 = dftrain2.loc[(IQR_outlier(dftrain2).loc[0, v] > dftrain2[v]) | (dftrain2[v] > IQR_outlier(dftrain2).loc[1, v]), :]
  
  outlier_table.at['정상 범위 데이터 개수', v] = temp1.shape[0]
  outlier_table.at['이상 범위 데이터 개수', v] = temp2.shape[0]
  outlier_table.at['전체 데이터 합계', v] = temp1.shape[0] + temp2.shape[0]

  temp3 = temp1.loc[dftrain2['Y_LABEL'] == 1, :]
  temp4 = temp2.loc[dftrain2['Y_LABEL'] == 1, :]
  outlier_table.at['정상 범위 데이터의 불량 개수', v] = temp3.shape[0]
  outlier_table.at['이상 범위 데이터의 불량 개수', v] = temp4.shape[0]
  outlier_table.at['불량 합계', v] = temp3.shape[0] + temp4.shape[0]  

  if temp1.shape[0] == 0 :
    temp5 = 0
  else :
    temp5 = np.round(temp3.shape[0] / temp1.shape[0] * 100, 3)
  outlier_table.at['정상 범위 데이터의 불량률', v] = temp5 

  if temp2.shape[0] == 0 :
    temp6 = 0
  else :
    temp6 = np.round(temp4.shape[0] / temp2.shape[0] * 100, 3)    
  outlier_table.at['이상 범위 데이터의 불량률', v] = temp6

outlier_table = outlier_table.T

In [ ]:
outlier_table.nlargest(len(variable), columns = ['이상 범위 데이터의 불량률'])

,정상 범위 데이터 개수,이상 범위 데이터 개수,전체 데이터 합계,정상 범위 데이터의 불량 개수,이상 범위 데이터의 불량 개수,불량 합계,정상 범위 데이터의 불량률,이상 범위 데이터의 불량률
AL,12651.0,1444.0,14095.0,506.0,697.0,1203.0,4.000,48.269
K,10738.0,1058.0,11796.0,854.0,141.0,995.0,7.953,13.327
FE,12397.0,1698.0,14095.0,1003.0,200.0,1203.0,8.091,11.779
TI,12802.0,1293.0,14095.0,1058.0,145.0,1203.0,8.264,11.214
MN,12498.0,1597.0,14095.0,1025.0,178.0,1203.0,8.201,11.146
SI,12246.0,1849.0,14095.0,1004.0,199.0,1203.0,8.199,10.763


In [ ]:
variable = variable

target_var = []

for v in variable :
  
  print(v)

  LB = IQR_outlier(dftrain2).loc[0, v]
  RB = IQR_outlier(dftrain2).loc[1, v]
  Defect_Q50 = dftrain2.loc[(dftrain2['Y_LABEL'] == 1) & (dftrain2[v].notnull()), v].quantile(0.5)
  
  print(LB, Defect_Q50, RB)

  if (Defect_Q50 < LB) or (Defect_Q50 > RB) :
    target_var.append(v)
  else :
    continue

AL
-3.5 22.0 8.5
MN
-3.0 1.0 5.0
FE
-173.5 58.0 326.5
K
-4.5 2.0 7.5
SI
-10.5 7.0 25.5
TI
0.0 0.0 0.0


In [ ]:
target_var

['AL']

In [ ]:
dftrain.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')

In [ ]:
dftrain

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,NaN,NaN,NaN,NaN,NaN,NaN,415,NaN,0.0,NaN,1,16,7,10,3,1,1175,1,645,9640,2,18,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,2244,NaN,0,0,170,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,NaN,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [ ]:
X=dftrain[['AL','MN','FE',"K", 'SI', 'TI']]
Y=dftrain['Y_LABEL']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0) 



In [ ]:
X_train

,AL,MN,FE,K,SI,TI
7460,0,6,52,1.0,5,1
8783,0,0,2,0.0,0,0
10688,4,9,340,NaN,25,0
1912,76,1,91,1.0,7,0
2433,47,10,874,2.0,11,0
...,...,...,...,...,...,...
13123,4,1,56,1.0,10,0
3264,4,62,499,3.0,21,0
9845,1,0,12,4.0,7,0
10799,0,1,55,0.0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
num= ['AL','MN','FE', 'SI', 'TI']
#숫자형 변수 - StandardScaler
#범주형 변수 -OneHotEncoder = 더미변수(가변수) 만드는


X = dftrain[num]
Y = dftrain['Y_LABEL']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:


#표준화, 원핫인코딩
ct = ColumnTransformer([('scling', StandardScaler(), num)])

ct.fit(X_train)
X_train= ct.transform(X_train)
X_test= ct. transform(X_test)

print(X_train[0])

[-0.13898558  0.4252785  -0.26391533 -0.15658052  0.05197873]


In [ ]:
smote=SMOTE(random_state=0)
X_train_over, Y_train_over = smote.fit_resample(X_train,Y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(C=1,random_state=0)

logit.fit(X_train_over, Y_train_over)
Y_pred=logit.predict(X_test)


#Y_pred
print("R2:{:.3f}".format(logit.score(X_train_over,Y_train_over)))

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))
print(logit.intercept_)
print(logit.coef_)

R2:0.770
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      3870
           1       0.50      0.58      0.54       359

    accuracy                           0.91      4229
   macro avg       0.73      0.76      0.75      4229
weighted avg       0.92      0.91      0.92      4229

[0.33890961]
[[10.80452641 -0.01737843  0.31724764 -0.44807222 -0.42607313]]


In [ ]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(C=1,random_state=0)

logit.fit(X_train_over, Y_train_over)
Y_pred=logit.predict(X_test)


#Y_pred
print("R2:{:.3f}".format(logit.score(X_train_over,Y_train_over)))

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))
print(logit.intercept_)
print(logit.coef_)

R2:0.770
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      3870
           1       0.50      0.58      0.54       359

    accuracy                           0.91      4229
   macro avg       0.73      0.76      0.75      4229
weighted avg       0.92      0.91      0.92      4229

[0.33890961]
[[10.80452641 -0.01737843  0.31724764 -0.44807222 -0.42607313]]


In [ ]:
from sklearn.linear_model import Ridge
lr_ridge = Ridge (random_state=0).fit(X_train, Y_train)
Y_pred=lr_ridge.predict(X_test)

print("예측값(Y predict value) \n",Y_pred)

예측값(Y predict value) 
 [0.07269397 0.08357872 0.06851364 ... 0.06846662 0.08239807 0.11338202]


In [ ]:
from sklearn.linear_model import Lasso
lr_lasso = Lasso(random_state=0, alpha = 0.01, max_iter=1000).fit(X_train, Y_train)
#max_iter=1000 - 계산의 횟수수
Y_pred=lr_lasso.predict(X_test)

print("예측값(Y predict value) \n",Y_pred)

예측값(Y predict value) 
 [0.07261765 0.08361441 0.06835086 ... 0.06830863 0.08198996 0.11443659]


In [ ]:
from scipy.stats.distributions import alpha
from sklearn.linear_model import ElasticNet
lr_ela = ElasticNet(alpha = 0.01).fit(X_train, Y_train)
#max_iter=1000 - 계산의 횟수수
Y_pred=lr_ela.predict(X_test)

print("예측값(Y predict value) \n",Y_pred)

예측값(Y predict value) 
 [0.07265531 0.08359391 0.06843431 ... 0.06838969 0.08219022 0.11395261]


In [ ]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(C=1,random_state=0)

logit.fit(X_train_over, Y_train_over)
Y_pred=logit.predict(X_test)


#Y_pred
print("R2:{:.3f}".format(logit.score(X_train_over,Y_train_over)))

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))
print(logit.intercept_)
print(logit.coef_)

R2:0.770
              precision    recall  f1-score   support

           0       0.99      0.03      0.05      3870
           1       0.09      1.00      0.16       359

    accuracy                           0.11      4229
   macro avg       0.54      0.51      0.11      4229
weighted avg       0.91      0.11      0.06      4229

[0.33890961]
[[10.80452641 -0.01737843  0.31724764 -0.44807222 -0.42607313]]


In [ ]:
#로지스틱회귀

#model
multilogit = LogisticRegression(random_state=0, C=0.1, solver='newton-cg',multi_class="multinomial")
multilogit.fit(X_train, Y_train)
Y_pred=multilogit.predict(X_test)

#성능
print("r2:{:.3f}".format(multilogit.score(X_train,Y_train)))
print(classification_report(Y_test,Y_pred))

r2:0.954
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      3870
           1       0.96      0.47      0.63       359

    accuracy                           0.95      4229
   macro avg       0.96      0.73      0.80      4229
weighted avg       0.95      0.95      0.95      4229



In [ ]:
#KNN 최근저이웃

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

scaler=StandardScaler().fit(X_train, Y_train)
X_test=scaler.transform(X_test)
X_train=scaler.transform(X_train)

X_train,Y_train=SMOTE(random_state=0).fit_resample(X_train,Y_train)

from sklearn.neighbors import KNeighborsClassifier
knn_model=KNeighborsClassifier(n_neighbors=9, metric="euclidean")

knn_model.fit(X_train,Y_train)
Y_pred=knn_model.predict(X_test)
print("accuracy:{:.3f}".format(knn_model.score(X_train,Y_train)))
print(classification_report(Y_test, Y_pred))

accuracy:0.896
              precision    recall  f1-score   support

           0       0.96      0.82      0.88      3870
           1       0.24      0.60      0.34       359

    accuracy                           0.80      4229
   macro avg       0.60      0.71      0.61      4229
weighted avg       0.90      0.80      0.84      4229

